In [1]:
%load_ext autoreload
%autoreload 2

import os

from process_events import build_events_df, build_charging_df

BASE_DATA_DIR = \
    "/Users/mt/workspace/ABMConsulting/LBNL/Code/beam/output/chargeTests/"

# Notes on household vehicle charging in BEAM

When electric vehicles plug in to recharge their electric "fuel" levels, they must search for a parking stall with a charging point. Each charging event is reflected in BEAM log files with three separate logged events: `ChargingPlugInEvent`, `RefuelSessionEvent`, and `ChargingPlugOutEvent`. How charging works and how vehicle state of charge (SOC) is initialized can be controlled by different configuration parameters reviewed below.  In this notebook I review (1) how to analyze charging data from BEAM log files; (2) how different charge configurations are specified; and (3) the effect of charge convergence on initial and final SOC across vehicles within and between iterations.


## Charging data

Charging data is composed of three event types: `ChargingPlugInEvent`, `RefuelSessionEvent`, and `ChargingPlugOutEvent`. A single `vehicle` participates in each event, which always occur together in succession. Like every BEAM event, these three events occur within a simulation `iter`(ation) and at a given `time` step within that iteration. `time` resets to 0 at the start of each iteration. The `ChargingPlugInEvent` occurs first, followed by refueling modeled as a `RefuelSessionEvent` 

Each charging event type occurs at a certain `chargingPointType` that indicates the type and power rating (e.g. `level2(7.2|AC)`), and a `parkingType` (`Residential`, `Workplace`, `Public`, etc.).


## Relevant configuration parameters

### Initial state of charge

Controlled using 

```
beam.agentsim.agents.vehicles.meanPrivateVehicleStartingSOC = 0.2
```

which in this case will set the initial charge to 20% of maximum on average across vehicles. If the initial charge is 80% or higher, charge will not be initiated overnight, resulting in log entries like the following where the `ChargingPlugOut` event occurs at the same time as `ChargingPlugIn`, and the fuel levels are identical for both events.

![image.png](attachment:image.png)



### Charge convergence

Used the following to enable conservation of charge which should cause initial charge and final charge to be the same. 

```
beam.sim.termination.terminateAtRideHailFleetStoredElectricityConvergence.minLastIteration = 1
beam.sim.termination.terminateAtRideHailFleetStoredElectricityConvergence.maxLastIteration = 10
beam.sim.termination.terminateAtRideHailFleetStoredElectricityConvergence.relativeTolerance = 0.01
```

`minLastIteration` is the minimum number of iterations to run the fixed point iteration to force charge convergence; `maxLastIteration` is the maximum number of iterations used to force charge convergence. If the stored electricity relative difference is less than `relativeTolerance` at any iteration greater than `minLastIteration` and less than `maxLastIteration` the algorithm has converged and iteration stops.

See [the TerminateAtRideHailFleetStoredElectricityConvergence.scala source code](https://github.com/LBNL-UCB-STI/beam/blob/develop/src/main/scala/beam/sim/termination/TerminateAtRideHailFleetStoredElectricityConvergence.scala) and this online resource (http://fourier.eng.hmc.edu/e176/lectures/NM/node17.html) for more information.

Charge convergence is difficult for the algorithm to compute without linking SOC across iterations, enabled by

```
beam.agentsim.agents.vehicles.linkSocAcrossIterations = true
```


## Charge convergence demonstration

Charge convergence should result in the start and end state of charge to be the same within each model iteration. We first group by the iteration and vehicle id, then summing over the minimum and maximum time present within the charging dataframe for the simulation. This implicitly selects ChargingPlugIn events for the start of iteration, and ChargingPlugOut events for the end of the iteration. 

It appears we need to use the storage capacity to do this calculation, as is done in `RideHailFleetStoredElectricityEventTracker.scala`:

```scala
val storedElectricityDifferenceInJoules =
        finalEvents.map(_.storedElectricityInJoules).sum - 
        initialEvents.map(_.storedElectricityInJoules).sum

val storageCapacityInJoules = initialEvents.map(_.storageCapacityInJoules).sum

storedElectricityDifferenceInJoules.abs / storageCapacityInJoules
```

However, it is not clear how to access this from the charging events. Apparently `capacity` is null in the datasets output on my system to the `events.csv(.gz)` files.

In [2]:
bv_chgcons_dir = os.path.join(
    BASE_DATA_DIR, "beamville_allparams__2022-03-02_14-51-43_wxe"
)
bv_chgcons = build_charging_df(bv_chgcons_dir)

bv_chgcons.capacity.isnull().all()

True

What about other non-charging events that may contain the charge capacity? 

In [3]:
bv_allevents = build_events_df(bv_chgcons_dir)
bv_allevents[(~bv_allevents.capacity.isnull())][['vehicle', 'capacity']]

,vehicle,capacity
2728,body-2,0.0
2731,2,4.0
2732,bus:B3-EAST-1-0,10.0
2733,bus:B3-WEST-1-0,10.0
2734,bus:B2-WEST-1-0,10.0
...,...,...
8954,bus:B1-WEST-1-191,10.0
8959,body-3,0.0
8961,rideHailVehicle-2@default,4.0
8963,rideHailVehicle-2@default,4.0


Apparently "capacity" refers to the number of passengers? But this is strange because there's also a `seatingCapacity` column. 

In [4]:
bv_allevents[(~bv_allevents.capacity.isnull())]\
            [['vehicle', 'capacity', 'seatingCapacity']].head(4)

,vehicle,capacity,seatingCapacity
2728,body-2,0.0,0.0
2731,2,4.0,4.0
2732,bus:B3-EAST-1-0,10.0,5.0
2733,bus:B3-WEST-1-0,10.0,5.0


OK, so it appears no information is available about charge capacity. We can still calcualte some version of relative difference for charge conversion checking. Specifically, we will calculate relative difference between initial and final charge within each iteration summed over all vehicles (call them `initialTotalCharge` and `finalTotalCharge`, respectively), normalized by their mean. We now calcualte this relative difference for the Beamville model run over five iterations with all charging configurations set as outlined above.

In [5]:
def reldiff(df):
    
    totcharge_start = df.groupby(['iter', 'vehicle']).min('time').groupby('iter').sum()
    totcharge_end = df.groupby(['iter', 'vehicle']).max('time').groupby('iter').sum()

    return abs(totcharge_start - totcharge_end) / ((totcharge_end + totcharge_start) / 2.0)

In [6]:
print(bv_chgcons.groupby(['iter', 'vehicle']).min('time').groupby('iter').sum()['primaryFuelLevel'])
print(bv_chgcons.groupby(['iter', 'vehicle']).max('time').groupby('iter').sum()['primaryFuelLevel'])
reldiff(bv_chgcons)

iter
0    6.217743e+08
1    7.653456e+08
2    7.937411e+08
3    7.915824e+08
4    7.893990e+08
5    7.923596e+08
Name: primaryFuelLevel, dtype: float64
iter
0    7.833058e+08
1    8.103023e+08
2    8.102988e+08
3    8.103010e+08
4    8.103033e+08
5    8.103023e+08
Name: primaryFuelLevel, dtype: float64


,time,primaryFuelLevel,capacity
iter,,,
0,2.0,0.229925,NaN
1,2.0,0.057064,NaN
2,2.0,0.020645,NaN
3,2.0,0.023371,NaN
4,2.0,0.026135,NaN
5,2.0,0.022391,NaN


There is significant charge convergence, mostly within 5% tolerance, but not within 1% tolerance as specified in our configuraion file. Maybe it is because there are few vehicles in Beamville. Let's try SF-light 0.5k.

In [7]:
sflight_chgcons_dir = os.path.join(
    BASE_DATA_DIR, "sflight_allparams__2022-03-02_21-24-55_ejx"
)
sflight_chgcons = build_charging_df(sflight_chgcons_dir)

In [8]:
print(sflight_chgcons.groupby(['iter', 'vehicle']).min('time').groupby('iter').sum()['primaryFuelLevel'])
print(sflight_chgcons.groupby(['iter', 'vehicle']).max('time').groupby('iter').sum()['primaryFuelLevel'])
reldiff(sflight_chgcons)

iter
0    4.353001e+10
1    8.166038e+10
2    8.334042e+10
3    8.367490e+10
4    8.368707e+10
5    8.378326e+10
Name: primaryFuelLevel, dtype: float64
iter
0    8.444513e+10
1    8.630988e+10
2    8.673727e+10
3    8.675217e+10
4    8.684335e+10
5    8.698694e+10
Name: primaryFuelLevel, dtype: float64


,time,primaryFuelLevel,capacity
iter,,,
0,2.0,0.639423,NaN
1,2.0,0.055361,NaN
2,2.0,0.039945,NaN
3,2.0,0.036112,NaN
4,2.0,0.037017,NaN
5,2.0,0.037520,NaN


As with Beamville, charge convergence is happening to within ~5% tolerance, revealed by the above analysis.